In [1]:
import dgl
import copy
import glob
import math
import torch
import pprint
import torch.nn as nn
import numpy as np
from os import path
from pathlib import Path
import matplotlib.pyplot as plt
from trainAndEval import train, evaluate
from dgl.data import DGLDataset
from TrainResults import TrainResults
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler
from CustomGCN_OnlyNFeatSumMsg import CustomGCN_OnlyNFeatSumMsg
from CustomGCN_OnlyNFeatMeanMsg import CustomGCN_OnlyNFeatMeanMsg
from ToyDGLDataset_v2 import ToyDGLDataset_v2, GetNodeFeatureVectors, GetEdgeFeatureVectors, GetNeighborNodes, GetEdgeList

plt.rcParams.update({'font.size': 20})
plt.rcParams['text.usetex'] = True
lw = 2
xyLabelFontSize = 20
xLabelPad = 10
yLabelPad = 15
pp = pprint.PrettyPrinter()

Using backend: pytorch


In [2]:
# training
def trainEpochs(model, device, dataloader, optimizer, loss_fn, batchsize, nEpochs):
    results = TrainResults()
    results.startTrainingTimer()
    bestModel = None
    bestValAcc = 0.0

    for epoch in range(nEpochs):
        # train
        epochLoss = train(model, device, dataloader, optimizer, loss_fn, batchsize, results)

        # evaluate
        train_result = evaluate(model, device, train_dataloader, loss_fn)
        val_result = evaluate(model, device, val_dataloader, loss_fn)
        test_result = evaluate(model, device, test_dataloader, loss_fn)

        results.addEpochResult(epochLoss, train_result, val_result, test_result)

        if(epoch % 5 == 0):
            results.printLastResult()

        if results.best_val_acc > bestValAcc:
            bestValAcc = results.best_val_acc
            bestModel = copy.deepcopy(model)
    
    results.endTrainingTimer()

    return results, bestModel

In [3]:
def getAllDatasetNames(datasetRootDir):
    files = glob.glob(datasetRootDir + '/*/*/*.json', recursive=True)
    #files = [x for x in files if "Toy2_v3_1" in x]
    files.sort()
    datasetDirectories = [path.dirname(file) for file in files]
    datasetnames = [path.normpath(dir).split(path.sep)[-1] for dir in datasetDirectories]
    return datasetDirectories, datasetnames

In [4]:
#device = 'cuda' if torch.cuda.is_available() else 'cpu'
device = 'cpu'

datasetRootDir = '/home/andrew/GNN_Sandbox/GraphToyDatasets_v3'
datasetDirs, datasetNames = getAllDatasetNames(datasetRootDir)

In [ ]:
import time
now = time.time()

batchSize = 1024
print(f'Device: {device}')

for i in range(len(datasetDirs)):
    dataset = ToyDGLDataset_v2(datasetNames[i], datasetDirs[i])
    splitIndices = dataset.get_split_indices()

    train_sampler = SubsetRandomSampler(splitIndices['train'])
    val_sampler = SubsetRandomSampler(splitIndices['valid'])
    test_sampler = SubsetRandomSampler(splitIndices['test'])

    train_dataloader = GraphDataLoader(dataset, sampler=train_sampler, batch_size=batchSize, drop_last=False)
    val_dataloader = GraphDataLoader(dataset, sampler=val_sampler, batch_size=batchSize, drop_last=False)
    test_dataloader = GraphDataLoader(dataset, sampler=test_sampler, batch_size=batchSize, drop_last=False)
    
    # Create the model with given dimensions
    model = CustomGCN_OnlyNFeatMeanMsg(dataset.dim_nfeats, 16, dataset.num_graph_classes).to(device)
    model.reset_parameters()
    optimizer = torch.optim.Adam(model.parameters(), lr=0.005)
    loss_fn = nn.CrossEntropyLoss()
    epochs = 30
    
    # train
    print(f'Beginning training on dataset {datasetNames[i]}')
    results, bestmodel = trainEpochs(model, device, train_dataloader, optimizer, loss_fn, batchSize, epochs)
    results.printBestResult()
    
    # save results
    outputFolder = path.join(path.join(datasetDirs[i], "ThesisPlots"), 'CustomGCN_OnlyNFeatMeanMsg')
    Path(outputFolder).mkdir(parents=True, exist_ok=True)
    
    results.savePlots(outputFolder)
    results.dumpSummary(outputFolder)
    results.pickledump(outputFolder)
    
    # save the best model for inference. (when loading for inference -> model.eval()!! )
    # https://pytorch.org/tutorials/beginner/saving_loading_models.html#what-is-a-state-dict
    torch.save(bestmodel.state_dict(), path.join(outputFolder, 'model.pt'))
    
    print(f'------------------({i+1}/{len(datasetDirs)}) models trained------------------\n')


end = time.time()
elapsed = end - now
print(f'{elapsed} seconds elapsed')

Device: cpu
Done loading data from cached files.
Beginning training on dataset Toy0_v3_1_0NodeCountMeanDiff
Epoch: 0, Loss: 0.8859, Validation Loss: 0.6937, Train: 0.501, Validation: 0.502, Test: 0.499, AUC: 0.493
Epoch: 5, Loss: 0.6945, Validation Loss: 0.6934, Train: 0.499, Validation: 0.501, Test: 0.496, AUC: 0.493
Epoch: 10, Loss: 0.6949, Validation Loss: 0.6931, Train: 0.499, Validation: 0.504, Test: 0.498, AUC: 0.493
Epoch: 15, Loss: 0.6957, Validation Loss: 0.6931, Train: 0.498, Validation: 0.502, Test: 0.500, AUC: 0.493
Epoch: 20, Loss: 0.6941, Validation Loss: 0.6938, Train: 0.501, Validation: 0.497, Test: 0.494, AUC: 0.493
Epoch: 25, Loss: 0.6938, Validation Loss: 0.6954, Train: 0.498, Validation: 0.502, Test: 0.506, AUC: 0.493
Best epoch: 
Epoch: 7, Loss: 0.6941, Validation Loss: 0.6931, Train: 0.499, Validation: 0.507, Test: 0.497, AUC: 0.493

The training took 168 seconds (2.80 minutes).
------------------(1/27) models trained------------------

Done loading data from cach

Epoch: 20, Loss: 0.3840, Validation Loss: 0.3743, Train: 0.841, Validation: 0.841, Test: 0.840, AUC: 0.924
Epoch: 25, Loss: 0.3619, Validation Loss: 0.3541, Train: 0.848, Validation: 0.847, Test: 0.846, AUC: 0.931
Best epoch: 
Epoch: 27, Loss: 0.3544, Validation Loss: 0.3480, Train: 0.848, Validation: 0.847, Test: 0.847, AUC: 0.932

The training took 196 seconds (3.27 minutes).
------------------(9/27) models trained------------------

Done loading data from cached files.
Beginning training on dataset Toy2_v3_0_0percentDiff
Epoch: 0, Loss: 0.7039, Validation Loss: 0.6960, Train: 0.497, Validation: 0.499, Test: 0.493, AUC: 0.490
Epoch: 5, Loss: 0.6969, Validation Loss: 0.6945, Train: 0.502, Validation: 0.496, Test: 0.493, AUC: 0.488
Epoch: 10, Loss: 0.6951, Validation Loss: 0.6933, Train: 0.501, Validation: 0.489, Test: 0.488, AUC: 0.488
Epoch: 15, Loss: 0.6953, Validation Loss: 0.6941, Train: 0.502, Validation: 0.497, Test: 0.494, AUC: 0.488
Epoch: 20, Loss: 0.6965, Validation Loss: 0.

Epoch: 5, Loss: 0.6608, Validation Loss: 0.6569, Train: 0.622, Validation: 0.625, Test: 0.625, AUC: 0.694
Epoch: 10, Loss: 0.6162, Validation Loss: 0.6100, Train: 0.682, Validation: 0.684, Test: 0.692, AUC: 0.765
Epoch: 15, Loss: 0.5636, Validation Loss: 0.5532, Train: 0.749, Validation: 0.747, Test: 0.756, AUC: 0.835
Epoch: 20, Loss: 0.5033, Validation Loss: 0.5026, Train: 0.775, Validation: 0.773, Test: 0.783, AUC: 0.883
Epoch: 25, Loss: 0.4385, Validation Loss: 0.4313, Train: 0.845, Validation: 0.843, Test: 0.848, AUC: 0.926
Best epoch: 
Epoch: 29, Loss: 0.3959, Validation Loss: 0.3895, Train: 0.865, Validation: 0.864, Test: 0.866, AUC: 0.951

The training took 250 seconds (4.17 minutes).
------------------(18/27) models trained------------------

Done loading data from cached files.
Beginning training on dataset Toy2_v3_1_0NodeCountMeanDiff
Epoch: 0, Loss: 1.0375, Validation Loss: 0.6941, Train: 0.499, Validation: 0.505, Test: 0.505, AUC: 0.507
Epoch: 5, Loss: 0.6950, Validation Lo

Best epoch: 
Epoch: 19, Loss: 0.5660, Validation Loss: 0.5594, Train: 0.696, Validation: 0.703, Test: 0.698, AUC: 0.782

The training took 195 seconds (3.25 minutes).
------------------(26/27) models trained------------------

Done loading data from cached files.
Beginning training on dataset Toy2_v3_1_9NodeCountMeanDiff
Epoch: 0, Loss: 0.9834, Validation Loss: 0.6963, Train: 0.497, Validation: 0.501, Test: 0.505, AUC: 0.502
Epoch: 5, Loss: 0.6946, Validation Loss: 0.6968, Train: 0.505, Validation: 0.502, Test: 0.497, AUC: 0.505
Epoch: 10, Loss: 0.6947, Validation Loss: 0.6934, Train: 0.422, Validation: 0.425, Test: 0.430, AUC: 0.505
Epoch: 15, Loss: 0.6946, Validation Loss: 0.6967, Train: 0.502, Validation: 0.498, Test: 0.494, AUC: 0.505
